In [17]:
import pandas as pd 
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [18]:
actor = pd.read_json('movieAppData/actor.json')
cast = pd.read_json('movieAppData/cast.json')
genres = pd.read_json('movieAppData/genres.json')
watchable = pd.read_json('movieAppData/watchable.json')

In [4]:
actor.head()

,id,bio,birth_date,first_name,image_path,last_name
0,1,"George Walton Lucas Jr. (born May 14, 1944) is...",1944-05-14,George,/WCSZzWdtPmdRxH9LUCVi2JPCSJ.jpg,Lucas
1,2,"Mark Richard Hamill (born September 25, 1951) ...",1951-09-25,Mark,/zMQ93JTLW8KxusKhOlHFZhih3YQ.jpg,Hamill
2,3,Legendary Hollywood Icon Harrison Ford was bor...,1942-07-13,Harrison,/ActhM39LTxgx3tnJv3s5nM6hGD1.jpg,Ford
3,4,Carrie Frances Fisher (21 October 1956 - 27 De...,1956-10-21,Carrie,/veakLIqGCbG0ek3YKfrlzcF72CG.jpg,Fisher
4,5,"Peter Wilton Cushing, OBE (26 May 1913 – 11 A...",1913-05-26,Peter,/if5g03wn6uvHx7F6FxXHLebKc0q.jpg,Cushing


In [5]:
cast.head()

,watchable_id,actor_id
0,646389,17276
1,646389,450
2,646389,1305610
3,646389,3417
4,646389,1257819


In [6]:
genres.head()

,watchable_id,genre
0,505642,ACTION
1,505642,ADVENTURE
2,505642,SCIENCE_FICTION
3,646389,ACTION
4,646389,ADVENTURE


In [7]:
watchable.head()

,id,description,duration,main_backdrop_path,name,poster_path,rating,release_date,trailer_link,type,vote_count
0,5,It's Ted the Bellhop's first night on the job....,98,/c1BaOxC8bo5ACFYkYYxL0bBWRaq.jpg,Four Rooms,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,5.727,1995-12-09,S_Pd2pGkq54,MOVIE,2307
1,12,"Nemo, an adventurous young clownfish, is unexp...",100,/yzqaKAhglTrkeOfuIXYYArf0WnA.jpg,Finding Nemo,/7DPFmp0q5RC1p582SdIuZ4KBF2.jpg,7.824,2003-05-30,ZS_8btMjx2U,MOVIE,17387
2,13,A man with a low IQ has accomplished great thi...,142,/3h1JZGDhZ8nzxdgvkxha0qBqi05.jpg,Forrest Gump,/arw2vcBveWOVZr6pxd9XTd1TdQa.jpg,8.479,1994-06-23,76WeEuR0qk4,MOVIE,24224
3,14,"Lester Burnham, a depressed suburban father in...",122,/DztBnZaqmla2sGUW9s8AyOmskT.jpg,American Beauty,/wby9315QzVKdW9BonAefg8jGTTb.jpg,8.031,1999-09-15,IJM3cZlqwlA,MOVIE,10837
4,15,"Newspaper magnate, Charles Foster Kane is take...",119,/3Ne0EpyuhZkIbfSyDzzMaNkb7Z7.jpg,Citizen Kane,/sav0jxhqiH0bPr2vZFU0Kjt2nZL.jpg,8.022,1941-04-17,fAcLNMkzfTE,MOVIE,4772


In [20]:
genres = genres.groupby("watchable_id").agg({'genre': list}).reset_index()
genres

,watchable_id,genre
0,5,"[CRIME, COMEDY]"
1,12,"[ANIMATION, FAMILY]"
2,13,"[COMEDY, DRAMA, ROMANCE, COMEDY]"
3,14,[DRAMA]
4,15,"[MYSTERY, DRAMA]"
...,...,...
13416,1087610,[DOCUMENTARY]
13417,1087771,"[ROMANCE, COMEDY]"
13418,1087819,"[THRILLER, DRAMA, TV_MOVIE]"
13419,1087828,"[DOCUMENTARY, TV_MOVIE, HORROR]"


In [21]:
watchable = watchable.merge(genres, how = 'left', left_on='id', right_on= 'watchable_id')

In [22]:
watchable

,id,description,duration,main_backdrop_path,name,poster_path,rating,release_date,trailer_link,type,vote_count,watchable_id,genre
0,5,It's Ted the Bellhop's first night on the job....,98,/c1BaOxC8bo5ACFYkYYxL0bBWRaq.jpg,Four Rooms,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,5.727,1995-12-09,S_Pd2pGkq54,MOVIE,2307,5.0,"[CRIME, COMEDY]"
1,12,"Nemo, an adventurous young clownfish, is unexp...",100,/yzqaKAhglTrkeOfuIXYYArf0WnA.jpg,Finding Nemo,/7DPFmp0q5RC1p582SdIuZ4KBF2.jpg,7.824,2003-05-30,ZS_8btMjx2U,MOVIE,17387,12.0,"[ANIMATION, FAMILY]"
2,13,A man with a low IQ has accomplished great thi...,142,/3h1JZGDhZ8nzxdgvkxha0qBqi05.jpg,Forrest Gump,/arw2vcBveWOVZr6pxd9XTd1TdQa.jpg,8.479,1994-06-23,76WeEuR0qk4,MOVIE,24224,13.0,"[COMEDY, DRAMA, ROMANCE, COMEDY]"
3,14,"Lester Burnham, a depressed suburban father in...",122,/DztBnZaqmla2sGUW9s8AyOmskT.jpg,American Beauty,/wby9315QzVKdW9BonAefg8jGTTb.jpg,8.031,1999-09-15,IJM3cZlqwlA,MOVIE,10837,14.0,[DRAMA]
4,15,"Newspaper magnate, Charles Foster Kane is take...",119,/3Ne0EpyuhZkIbfSyDzzMaNkb7Z7.jpg,Citizen Kane,/sav0jxhqiH0bPr2vZFU0Kjt2nZL.jpg,8.022,1941-04-17,fAcLNMkzfTE,MOVIE,4772,15.0,"[MYSTERY, DRAMA]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14254,1087610,"For over forty years, Jacques Duhoux, expediti...",90,/JQdfc4weBMeqc8iS88qpMEqPBl.jpg,Jacques,/juUHTuj4z3wsAoUlPaBsmYCsBdB.jpg,0.000,2023-02-26,,MOVIE,0,1087610.0,[DOCUMENTARY]
14255,1087771,"After Carlos is tasked to care for Paola, both...",90,/h4UJHYhIa80kpN4Ak6rmBQN1C2r.jpg,Nada Que Ver,/ofnOwcG9l1DuGl7vB45JHsfSlR6.jpg,2.000,2023-03-02,,MOVIE,1,1087771.0,"[ROMANCE, COMEDY]"
14256,1087819,A young woman’s dad becomes the obsession of h...,90,/wYMMcpRsT8D7jvz1ThIfzqIGebz.jpg,Her Affair to Die For,/5y1BMHtomCuZ9cUtD0NTTmqppT1.jpg,9.000,2023-02-23,,MOVIE,2,1087819.0,"[THRILLER, DRAMA, TV_MOVIE]"
14257,1087828,Reveals the shocking haunting that took over M...,88,/4uA2ks2hBPn8hjo4i4CUFBI1cgr.jpg,The Devil's Academy,/f6hvJiLeyyqYUcBQRL6FF54Brq.jpg,0.000,2023-02-05,,MOVIE,0,1087828.0,"[DOCUMENTARY, TV_MOVIE, HORROR]"


In [23]:
watchable.shape

(14259, 13)

In [24]:
watchable.isna().sum()

id                      0
description             0
duration                0
main_backdrop_path      0
name                    0
poster_path             0
rating                  0
release_date            0
trailer_link            0
type                    0
vote_count              0
watchable_id          838
genre                 838
dtype: int64

In [25]:
cast = cast.merge(actor[['id', "first_name", "last_name"]],how = 'left', left_on = 'actor_id', right_on = 'id')

In [26]:
cast['actor name'] = cast['first_name'] + " " + cast['last_name']

In [27]:
cast_new = cast.groupby("watchable_id").agg({"actor name": list}).reset_index()

In [28]:
cast_new

,watchable_id,actor name
0,5,"[Tim Roth, Jennifer Beals, Antonio Banderas, V..."
1,12,"[Albert Brooks, Ellen DeGeneres, Alexander Gou..."
2,13,"[Tom Hanks, Robin Wright, Gary Sinise, Mykelti..."
3,14,"[Kevin Spacey, Annette Bening, Thora Birch, We..."
4,15,"[Fred Rogers, Betty Aberlin, David Newell, Joe..."
...,...,...
13500,1087061,"[Jenna Katheryn, Duncan Sage]"
13501,1087771,"[Memo Villegas, Fiona Palomo, Nora Velázquez, ..."
13502,1087819,"[Logan Mariner, Ryan Francis, Sean Benum, Mike..."
13503,1087828,"[Gustavo Benajam, Frank Lopez Del Rincon, Lynn..."


In [29]:
watchable = watchable.merge(cast_new, left_on = 'id', right_on = 'watchable_id')

In [30]:
watchable.columns

Index(['id', 'description', 'duration', 'main_backdrop_path', 'name',
       'poster_path', 'rating', 'release_date', 'trailer_link', 'type',
       'vote_count', 'watchable_id_x', 'genre', 'watchable_id_y',
       'actor name'],
      dtype='object')

In [36]:
def content_based_filtering(movie_title, data = watchable.iloc[:1000], n_recommendations=5):
    """
    Recommends movies based on their genres, actors, descriptions, and titles using cosine similarity.

    Parameters:
        - movie_title: str, title of the movie for which we want to recommend similar movies
        - data: pd.DataFrame, dataframe containing movie titles, genres, actors, and descriptions
        - n_recommendations: int, number of recommendations to return (default=5)

    Returns:
        - recommendations: pd.DataFrame, dataframe containing recommended movies
    """
    data['actor name'] = data['actor name'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    data['genre'] = data['genre'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

    # Compute the tf-idf matrix for the movie descriptions
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(data['description'])

    # Compute the cosine similarity matrix between movies based on descriptions
    cosine_sim_matrix_desc = cosine_similarity(tfidf_matrix)

    # Compute the cosine similarity matrix between movies based on genres and actors
    genres_actors_matrix = data[['genre', 'actor name']]
    genres_actors_matrix = pd.get_dummies(genres_actors_matrix)
    cosine_sim_matrix_ga = cosine_similarity(genres_actors_matrix)

    # Find the index of the movie in the dataframe
    movie_index = data[data['name'] == movie_title].index[0]

    # Compute the weighted cosine similarity between the movie and all other movies
    sim_scores = (0.5 * cosine_sim_matrix_desc[movie_index]) + (0.5 * cosine_sim_matrix_ga[movie_index])

    # Sort the movies by their similarity score
    sim_scores = list(enumerate(sim_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top recommendations
    top_indices = [i[0] for i in sim_scores[1:n_recommendations+1]]

    # Get the recommended movies from the dataframe
    recommendations = data.iloc[top_indices][['name', 'genre', 'actor name', 'description']].reset_index(drop=True)

    return recommendations


In [37]:
content_based_filtering("Harry Potter and the Philosopher's Stone")

,name,genre,actor name,description
0,Harry Potter and the Chamber of Secrets,"ADVENTURE, FANTASY","Daniel Radcliffe, Rupert Grint, Emma Watson, K...","Cars fly, trees fight back, and a mysterious h..."
1,Harry Potter and the Prisoner of Azkaban,"ADVENTURE, FANTASY","Daniel Radcliffe, Rupert Grint, Emma Watson, R...",Year three at Hogwarts means new fun and chall...
2,Harry Potter and the Half-Blood Prince,"ADVENTURE, FANTASY","Daniel Radcliffe, Rupert Grint, Emma Watson, H...",As Lord Voldemort tightens his grip on both th...
3,Harry Potter and the Order of the Phoenix,"ADVENTURE, FANTASY, MYSTERY","Daniel Radcliffe, Rupert Grint, Emma Watson, H...",Returning for his fifth year of study at Hogwa...
4,Harry Potter and the Goblet of Fire,"ADVENTURE, FANTASY, FAMILY","Daniel Radcliffe, Rupert Grint, Emma Watson, R...",When Harry Potter's name emerges from the Gobl...


In [36]:
content_based_filtering("Inception")

,name,genre,actor name,description
0,The Tomorrow War,"ACTION, SCIENCE_FICTION, ADVENTURE","Chris Pratt, Yvonne Strahovski, J.K. Simmons, ...",The world is stunned when a group of time trav...
1,Alita: Battle Angel,"ACTION, SCIENCE_FICTION, ADVENTURE","Rosa Salazar, Christoph Waltz, Jennifer Connel...",When Alita awakens with no memory of who she i...
2,Dune,"ACTION, SCIENCE_FICTION, ADVENTURE","Kyle MacLachlan, Francesca Annis, Patrick Stew...","In the year 10,191, the world is at war for co..."
3,Moontrap: Target Earth,"ACTION, SCIENCE_FICTION, ADVENTURE","Sarah Butler, Damon Dayoub, Charles Shaughness...",A long forgotten ancient spacecraft discovered...
4,Transformers: Dark of the Moon,"ACTION, SCIENCE_FICTION, ADVENTURE","Shia LaBeouf, Rosie Huntington-Whiteley, Josh ...","The Autobots continue to work for NEST, now no..."


In [37]:
content_based_filtering("Interstellar")

,name,genre,actor name,description
0,The Mysterious Cities of Gold,"ANIMATION, ACTION, ADVENTURE, SCIENCE_FICTION,...","Véronique Desmadryl, Caroline Mozzone, Adelin...",The adventures of a young Spanish boy named Es...
1,Space Pirate Captain Harlock,"ANIMATION, SCIENCE_FICTION","Shun Oguri, Haruma Miura, Yu Aoi, Arata Furuta...",Space Pirate Captain Harlock and his fearless ...
2,Prometheus,"SCIENCE_FICTION, ADVENTURE, MYSTERY, SCIENCE_F...","Noomi Rapace, Michael Fassbender, Logan Marsha...",A team of explorers discover a clue to the ori...
3,Skylanders Academy,"ACTION, ADVENTURE, ANIMATION, FAMILY","Justin Long, Ashley Tisdale, Jonathan Banks, C...",Follow the heroic adventures of the Skylanders...
4,The Deep,"FAMILY, ANIMATION, ACTION, ADVENTURE","Vincent Tong, Ashleigh Ball, Michael Dobson, K...","The adventures of the Nekton family, a family ..."


In [38]:
content_based_filtering("Avengers: Endgame")

,name,genre,actor name,description
0,Valerian and the City of a Thousand Planets,"ADVENTURE, SCIENCE_FICTION, ACTION","Dane DeHaan, Cara Delevingne, Clive Owen, Riha...","In the 28th century, Valerian and Laureline ar..."
1,Transformers,"ADVENTURE, SCIENCE_FICTION, ACTION","Shia LaBeouf, Megan Fox, Josh Duhamel, Tyrese ...",Young teenager Sam Witwicky becomes involved i...
2,Starship Troopers 3: Marauder,"ADVENTURE, SCIENCE_FICTION, ACTION","Casper Van Dien, Jolene Blalock, Stephen Hogan...",The war against the Bugs continues! A Federati...
3,LEGO Marvel Super Heroes: Black Panther - Trou...,"ANIMATION, FANTASY, FAMILY","James C. Mathis III, Keston John, Yvette Nicol...","After speaking at the Wakanda Embassy, Black P..."
4,Avengers: Infinity War,"ADVENTURE, ACTION, SCIENCE_FICTION","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",As the Avengers and their allies have continue...
